# Actor-Critic Methods

```{note}
In Policy-Based methods, we aim to optimize the policy directly without using a value function. We saw that Reinforce worked well. However, because we use Monte-Carlo sampling to estimate return, we have significant variance in policy gradient estimation.<br>
So today we’ll study Actor-Critic methods, a hybrid architecture combining value-based and Policy-Based methods that helps to stabilize the training by reducing the variance using:<br>
* An Actor that controls how our agent behaves (Policy-Based method)<br>
* A Critic that measures how good the taken action is (Value-Based method)
```

## The Problem of Variance in Reinforce

In Reinforce, we want to increase the probability of actions in a trajectory proportionally to how high the return is.

![](images/actor1.jpeg)

This return $R(\tau)$ is calculated using a Monte-Carlo sampling. We collect a trajectory and calculate the discounted return, and use this score to increase or decrease the probability of every action taken in that trajectory.

The advantage of this method is that it’s unbiased. Since we’re not estimating the return, we use only the true return we obtain.

Given the stochasticity of the environment and stochasticity of the policy, trajectories can lead to different returns, which can lead to high variance. Consequently, the same starting state can lead to very different returns. Because of this, the return starting at the same state can vary significantly across episodes.

![](images/actor2.jpeg)

The solution is to mitigate the variance by using a large number of trajectories. However, increasing the batch size significantly reduces sample efficiency. So we need to find additional mechanisms to reduce the variance.

## Advantage Actor-Critic (A2C)

### Reducing variance with Actor-Critic methods

The solution to reducing the variance of the Reinforce algorithm and training our agent faster and better is to use a combination of Policy-Based and Value-Based methods: the Actor-Critic method.

To understand the Actor-Critic, imagine you’re playing a video game. You can play with a friend that will provide you with some feedback. You’re the Actor and your friend is the Critic.

![](images/ac3.jpeg)

You don’t know how to play at the beginning, so you try some actions randomly. The Critic observes your action and provides feedback. Learning from this feedback, you’ll update your policy and be better at playing that game. On the other hand, your friend (Critic) will also update their way to provide feedback so it can be better next time.

This is the idea behind Actor-Critic. We learn two function approximations:

* A policy that controls how our agent acts: $\pi_{\theta}(s)$

* A value function to assist the policy update by measuring how good the action taken is: $\hat{q}_{w}(s, a)$

### The Actor-Critic Process

Let’s see the training process to understand how the Actor and Critic are optimized:

* At each timestep $t$, we get the current state $S_{t}$ from the environment and pass it as input through our Actor and Critic.

* Our Policy takes the state and outputs an action $A_{t}$.

* The Critic takes that action also as input and, using $S_{t}$ and $A_{t}$, computes the value of taking that action at that state: the Q-value.

* The action $A_{t}$ performed in the environment outputs a new state $S_{t+1}$ and a reward $R_{t+1}$.

* The Actor updates its policy parameters using the Q value.

![](images/ac4.jpeg)

* Thanks to its updated parameters, the Actor produces the next action to take at $A_{t+1}$ given the new state $S_{t+1}$.

* The Critic then updates its value parameters.

$$\Delta{w} = \beta(r_{t+1} + \gamma\hat{q}_{w}(s_{t+1}, a_{t+1}) - \hat{q}_{w}(s_{t}, a_{t}))\nabla_{w}\hat{q}_{w}(s_{t}, a_{t})$$

```{tip}
If we fix the TD target.

$$\Delta{w} = \frac{\beta}{2}\nabla_{w}(r_{t+1} + \gamma\hat{q}_{w}(s_{t+1}, a_{t+1}) - \hat{q}_{w}(s_{t}, a_{t}))^{2}$$
```

### Adding Advantage in Actor-Critic (A2C)

We can stabilize learning further by using the Advantage function as Critic instead of the Action value function.

The idea is that the Advantage function calculates the relative advantage of an action compared to the others possible at a state: how taking that action at a state is better compared to the average value of the state. It’s subtracting the mean value of the state from the state action pair:

![](images/ac5.jpeg)

The extra reward is what’s beyond the expected value of that state.

* If $A(s, a) > 0$, our gradient is pushed in that direction.

* If $A(s, a) < 0$, our gradient is pushed in the opposite direction.

The problem with implementing this advantage function is that it requires two value functions - $Q(s, a)$ and $V(s)$, Fortunately, we can use the TD error as a good estimator of the advantage function.

![](images/ac6.jpeg)